In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
new_article_pattern  = '^Copyright \(c\) \d\d\d\d The Kansas City Star\n$'
title_pattern = '\| Kansas City Star, The \(MO\)\n$'
author_pattern = 'OpenURL Link\n$'
signature_pattern = ': \d\d\d\-\d\d\d-\d\d\d\d(,)?(\son\sTwitter)?(\s@\w+)?$'

In [3]:
# Make sure file gets closed after being iterated
with open('../data/KC_STAR/kc_star.txt', 'r') as f:
   # Read the file contents and generate a list with each line
   lines = f.readlines()

In [4]:
k = 0
for line in lines:
    k += (re.match(new_article_pattern, line)!=None)
print(k)

1000


In [5]:
articles = []
j = 0
for i in range(k):    
    title = ''
    date = ''
    author = ''
    text = ''
    
    while re.search(title_pattern, lines[j]) == None:
        title += " " + lines[j].strip()
        j += 1
        
    date = lines[j].strip()
    date = re.sub(r' \| Kansas City Star, The.*', '', date).strip()
    j += 1
    
    while re.search(author_pattern, lines[j]) == None:
        author += lines[j].strip()
        j += 1
    author += lines[j].strip()
    author = re.sub('Author: ', '', author).strip()
    author = re.sub(r'Section: ?8?1?6? ?n?o?r?t?h? ?business.*', '', author).strip()
    j += 1
    
    while re.search(new_article_pattern, lines[j]) == None:
        if re.search(signature_pattern, lines[j]) == None: 
            if re.search('^The Star\n$', lines[j]) == None:
                text += lines[j].strip() + " "
        j = j+1        
    j = j+1
    article = [title.strip(), date.strip(), author.strip(), text.strip()]
    articles.append(article)
    i += 1

In [6]:
df = pd.DataFrame(np.array(articles).reshape(k,4), columns = ['title', 'date', 'author', 'text'])

In [7]:
df.shape

(1000, 4)

In [8]:
df = df.drop_duplicates('text').reset_index()

In [9]:
df.shape

(999, 5)

In [10]:
df['text'] = df['text'].str.replace('’', ' ')
df['text'] = df['text'].str.replace('\'', ' ')
df['text'] = df['text'].str.replace('-', ' ')

In [11]:
df.to_csv('../data/KC_STAR/kc_star.csv', index = False)